In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from collections import Counter
import geopy.distance
from geopy.geocoders import Nominatim
import tkinter as tk
from tkinter import *
from tkinter.ttk import *
from tkinter import messagebox

def euclidian_distance(x1, x2):
    val = np.sqrt(np.sum((x1-x2)**2))
    return val

def manhattan_distance(x1, x2):
    val = np.sum(np.absolute(x1-x2))
    return val

k = -1;

In [2]:
class CustomkNN:
    
    def __init__(self, k=3):
        self.k = k
    
    #fit training labels and training set
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
    
    
    def predict(self, X, type_distance = 0): #can get multiple samples
        if type_distance == 1:
            print("Manhattan")
        else:
            print("Euclidian")
            
        print(X)
        predicted_labels = [self._predict(x, type_distance) for x in X]
        print(predicted_labels)
        return np.array(predicted_labels)
        
    def _predict(self, x, type_distance = 0):
        #racuna distancu
        if type_distance == 0 :  #euclidian distance
            distances = [euclidian_distance(x, x_train) for x_train in self.X_train.values]
        elif type_distance == 1:
            distances = [manhattan_distance(x, x_train) for x_train in self.X_train.values]
            
        #odredjuje k najblizih suseda
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        
        #odredjuje klasu
        most_common = Counter(k_nearest_labels).most_common(1)
        class_num = most_common[0][0]
        decided_class = decide_class(class_num)
        print(decided_class)
        
        return class_num

In [3]:
def define_class(y):
    if y <= 49999:
        return 0
    elif (y >= 50000) and (y <= 99999):
        return 1
    elif (y >= 100000) and (y <= 149999):
        return 2
    elif (y >= 150000) and (y <= 199999):
        return 3
    else:
        return 4
    
def decide_class (class_num):
    if class_num == 0:
        return "price <= 49999"
    elif class_num == 1:
        return "50000 <= price <= 99999"
    elif class_num == 2:
        return "100000 <= price <= 149999"
    elif class_num == 3:
        return "150000 <= price <= 199999"
    else :
        return "price >= 200000"

def return_data() :
    #ucitavanje podataka
    X = pd.read_csv("with_distance.csv")
    print(X.dtypes)
    Y = pd.read_csv("y_non_null.csv")

    X = X[['broj_soba', 'kvadratura_m2', 'spratnost', 'udaljenost', 'godina']]
    Y = Y['cena_eur']
    Y = [define_class(y) for y in Y.values]
    print(Y)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1234)
    k = np.sqrt(X_train.shape[0])
    print(k)
    
    X_train = X_train[['broj_soba', 'kvadratura_m2', 'spratnost', 'udaljenost', 'godina']]
    return X_train,X_test,Y_train, Y_test

def distanca_od_centra_km(lokacija):
    print(lokacija)
    geolocator = Nominatim(user_agent="MOJ_APP")
    #koordinate centra
    location = geolocator.geocode("Kneza Mihaila Beograd")
    coords_centar = (location.latitude, location.longitude)
    print(coords_centar)
    #koordinate lokacija
    location = geolocator.geocode(lokacija)
    if location != None:
        coords_lokacija = (location.latitude, location.longitude)
        print(coords_lokacija)
        distanca = geopy.distance.distance(coords_lokacija, coords_centar).km
        print(distanca)
        return distanca
    else:
        return None

def test_knn(x, k = 3, type_distance = 0 ):
    X_train, X_test, Y_train, Y_test = return_data()
    clf = None;
    num_sim = k;
    if num_sim == -1:
        num_sim = int(np.sqrt(X_train.shape[0]))
    clf = CustomkNN(k = num_sim)
    clf.fit(X_train, Y_train)
    if x.empty:
        predictions = clf.predict(X_test.values, type_distance)
    else:
        print(x)
        x = x[['broj_soba', 'kvadratura_m2', 'spratnost', 'udaljenost', 'godina']]
        predictions = clf.predict(x.values, type_distance)
    
    return predictions;
    
def test_knn_gui():
    def activator (): 
        if k_label['state'] == tk.DISABLED and e6['state'] == tk.DISABLED:
            k_label['state'] = tk.NORMAL
            e6['state'] = tk.NORMAL
        else:
            k_label['state'] = tk.DISABLED
            e6['state'] = tk.DISABLED
            
    window = tk.Tk()
    window.title("KNN algorithm test")
    window.geometry('400x200')
    
    frame = tk.Frame(window, relief='sunken', bg = 'white')
    frame.pack(fill= BOTH, expand=True, padx=10, pady = 20)
    broj_soba_label = tk.Label(frame, text="Broj soba")
    broj_soba_label.grid(row=1, column=1)
    
    kvadratura_m2_label = tk.Label(frame, text="Kvadratura(m2)")
    kvadratura_m2_label.grid(row=3, column=1)
    
    sprat_label = tk.Label(frame, text="Sprat")
    sprat_label.grid(row=5, column=1)
    
    lokacija_label = tk.Label(frame, text="Lokacija")
    lokacija_label.grid(row=7, column=1)
    
    godina_label = tk.Label(frame, text="Godina")
    godina_label.grid(row=9, column=1)
    
    man = tk.IntVar() #0 unchecked 1 checked
    chk_Manhattan = tk.Checkbutton(frame, text="Manhattan", variable = man)
    chk_Manhattan.grid(row = 11, column=1)
    
    euk = tk.IntVar() #0 unchecked 1 checked
    chk_Euclidian = tk.Checkbutton(frame, text="Euclidian", variable = euk)
    chk_Euclidian.grid(row = 11, column=2)
    
    enter_k_chk = tk.Checkbutton(frame, text="Unesi parametar", highlightbackground='black', font="Arial, 8", command=activator)
    enter_k_chk.grid(row = 13, column=1)
    
    k_label = tk.Label(frame, text="K parametar", state = tk.DISABLED)
    k_label.grid(row=15, column=1)
    
    
    e1 = tk.Entry(frame)
    e2 = tk.Entry(frame)
    e3 = tk.Entry(frame)
    e4 = tk.Entry(frame)
    e5 = tk.Entry(frame)
    e6 = tk.Entry(frame, state=tk.DISABLED)
    e1.grid(row=1, column=2)
    e2.grid(row=3, column=2)
    e3.grid(row=5, column=2)
    e4.grid(row=7, column=2)
    e5.grid(row=9, column=2)
    e6.grid(row=15, column=2)
    
    # kreira objekat Style
    style = Style()

    def clicked(broj_soba, kvadratura, sprat, lokacija, godina, manhattan, euclidian, k=-1):
        if broj_soba == None or broj_soba == "" or kvadratura == None or kvadratura == "" or sprat == None or sprat == "" or lokacija == None or lokacija == "" or godina == None or godina == "":
            tk.messagebox.showerror("Error", "Unesite sve podatke!")
        else :
            try:
                soba = int(broj_soba)
                print(type(soba))
                kvadratura_m2 = int(kvadratura)
                print(type(kvadratura_m2))
                ukupno_spratova = int(sprat)
                print(type(ukupno_spratova))
                print(type(lokacija))
                distanca = distanca_od_centra_km(lokacija + " Beograd")
                print(distanca)
                godina_int = int(godina)
                print(type(godina))
            
                test = {'broj_soba': [soba], 'kvadratura_m2': [kvadratura_m2], 'spratnost': [ukupno_spratova], 'udaljenost':[distanca], 'godina': [2020]}
                test_df = pd.DataFrame(test)
            
                k = -1
                if k_label['state'] == tk.NORMAL:
                    k = int(e6.get())
                type_dist = 0;
                if man == 1:
                    type_dist = 1
            
                predvidjena_klasa = test_knn(test_df, k, type_distance = type_dist)[0]
                tk.messagebox.showinfo("Results", "Predvidjena klasa: " + decide_class(predvidjena_klasa))
            except Exception as error:
                print(error)
                tk.messagebox.showerror("Error", "Pokusajte ponovo!")
                
    # Dodaje definisan stil na dugmetu
    style.configure('W.TButton', font =
               ('calibri', 10, 'bold', 'underline'),
                foreground = 'red')
    btn = Button(frame, text="Predvidi opseg", state="DISABLED" , style="W.TButton", command=(lambda: clicked(e1.get(), e2.get(), e3.get(), e4.get(), e5.get(), man, euk, k)))
    btn.grid(row = 17, column=1)
    
    window.mainloop()

In [4]:
#test knn / MAIN part
test_knn_gui()